In [1]:
import pandas as pd
from sodapy import Socrata
import numpy as np
import pymongo
import datetime
from flask import jsonify
from bson.json_util import dumps, loads
import requests
import json

In [2]:
# To display all the columns
pd.set_option('display.max_columns', None)
# To display all the rows
# pd.set_option('display.max_rows', None)

In [3]:
client = Socrata("data.sfgov.org", None)
results = client.get("wg3w-h783",incident_year="2020", limit=100000)

In [4]:
results_df = pd.DataFrame.from_records(results)
results_df.head()

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,report_type_code,report_type_description,filed_online,incident_code,incident_category,incident_subcategory,incident_description,resolution,intersection,cnn,police_district,analysis_neighborhood,supervisor_district,latitude,longitude,point,cad_number
0,2020-02-25T00:00:00.000,2020-02-25T00:00:00.000,00:00,2020,Tuesday,2020-02-26T18:11:00.000,90891206224,908912,206046852,II,Coplogic Initial,True,06224,Larceny Theft,Larceny - From Vehicle,"Theft, From Unlocked Vehicle, >$950",Open or Active,21ST ST \ ALABAMA ST,23996000,Mission,Mission,9,37.75748490120071,-122.41142437143391,"{'type': 'Point', 'coordinates': [-122.4114243...",NaN
1,2020-03-03T09:30:00.000,2020-03-03T00:00:00.000,09:30,2020,Tuesday,2020-03-03T11:15:00.000,90903707045,909037,200158411,VS,Vehicle Supplement,NaN,07045,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Truck",Open or Active,03RD ST \ HOLLISTER AVE,20471000,Southern,Bayview Hunters Point,10,37.72171587946975,-122.39594382884452,"{'type': 'Point', 'coordinates': [-122.3959438...",200623433
2,2020-03-03T12:11:00.000,2020-03-03T00:00:00.000,12:11,2020,Tuesday,2020-03-03T14:15:00.000,90916075000,909160,200160472,II,Initial,NaN,75000,Missing Person,Missing Person,Found Person,Open or Active,LARKIN ST \ BUSH ST,25204000,Northern,Nob Hill,3,37.78890961112855,-122.41865587356023,"{'type': 'Point', 'coordinates': [-122.4186558...",200632333
3,2020-03-03T05:00:00.000,2020-03-03T00:00:00.000,05:00,2020,Tuesday,2020-03-03T10:11:00.000,90902727195,909027,200159691,II,Initial,NaN,27195,Other Miscellaneous,Trespass,Trespassing,Open or Active,GREEN ST \ JONES ST,25271000,Central,Russian Hill,3,37.79868012645644,-122.41557232083188,"{'type': 'Point', 'coordinates': [-122.4155723...",200631201
4,2020-03-02T08:04:00.000,2020-03-02T00:00:00.000,08:04,2020,Monday,2020-03-02T08:07:00.000,90902107045,909021,200155768,VS,Vehicle Supplement,NaN,07045,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Truck",Open or Active,BRYANT ST \ LANGTON ST,23876000,Southern,South of Market,6,37.77385630397913,-122.40528580238285,"{'type': 'Point', 'coordinates': [-122.4052858...",200630723


In [5]:
results_df["incident_category"].unique()

array(['Larceny Theft', 'Recovered Vehicle', 'Missing Person',
       'Other Miscellaneous', 'Warrant', 'Robbery', 'Malicious Mischief',
       'Non-Criminal', 'Assault', 'Other', 'Weapons Offense',
       'Traffic Violation Arrest', 'Motor Vehicle Theft',
       'Miscellaneous Investigation', 'Lost Property',
       'Offences Against The Family And Children', 'Burglary',
       'Disorderly Conduct', 'Suspicious Occ', 'Embezzlement',
       'Case Closure', 'Arson', 'Traffic Collision', 'Drug Offense',
       'Stolen Property', 'Vehicle Impounded', 'Weapons Carrying Etc',
       'Drug Violation', 'Fraud', 'Courtesy Report',
       'Forgery And Counterfeiting', 'Weapons Offence', 'Sex Offense',
       'Family Offense', 'Vehicle Misplaced', 'Other Offenses',
       'Fire Report', 'Civil Sidewalks', 'Vandalism', nan, 'Prostitution',
       'Suspicious', 'Liquor Laws', 'Suicide', 'Homicide', 'Rape',
       'Human Trafficking (A), Commercial Sex Acts'], dtype=object)

In [6]:
results_df["incident_category"].value_counts()

Larceny Theft                                 7457
Other Miscellaneous                           1720
Malicious Mischief                            1489
Non-Criminal                                  1374
Assault                                       1324
Burglary                                      1000
Motor Vehicle Theft                            915
Warrant                                        891
Recovered Vehicle                              786
Lost Property                                  693
Drug Offense                                   641
Robbery                                        631
Fraud                                          511
Missing Person                                 477
Offences Against The Family And Children       435
Disorderly Conduct                             402
Suspicious Occ                                 394
Traffic Violation Arrest                       316
Miscellaneous Investigation                    194
Other Offenses                 

In [7]:
result_dropped_columns = results_df.drop(['incident_date','report_datetime','row_id','incident_id',
                                          'incident_number','report_type_code','report_type_description',
                                          'filed_online','incident_code','incident_subcategory','cad_number',
                                          'intersection','cnn','analysis_neighborhood','supervisor_district',
                                          'point'], axis=1)
# result_dropped_columns

In [8]:
result_dropped_columns = result_dropped_columns.dropna()

In [9]:
result_renamed = result_dropped_columns.rename(columns={'incident_datetime':'Datetime','incident_time':'Time',
                                         'incident_year':'Year','incident_day_of_week':'Week','incident_category':'Category',
                                         'incident_description':'Description','resolution':'Resolution',
                                         'police_district':'Police_Dist','latitude':'Lat','longitude':'Lon'})

In [10]:
result_renamed.head()

,Datetime,Time,Year,Week,Category,Description,Resolution,Police_Dist,Lat,Lon
0,2020-02-25T00:00:00.000,00:00,2020,Tuesday,Larceny Theft,"Theft, From Unlocked Vehicle, >$950",Open or Active,Mission,37.75748490120071,-122.41142437143391
1,2020-03-03T09:30:00.000,09:30,2020,Tuesday,Recovered Vehicle,"Vehicle, Recovered, Truck",Open or Active,Southern,37.72171587946975,-122.39594382884452
2,2020-03-03T12:11:00.000,12:11,2020,Tuesday,Missing Person,Found Person,Open or Active,Northern,37.78890961112855,-122.41865587356023
3,2020-03-03T05:00:00.000,05:00,2020,Tuesday,Other Miscellaneous,Trespassing,Open or Active,Central,37.79868012645644,-122.41557232083188
4,2020-03-02T08:04:00.000,08:04,2020,Monday,Recovered Vehicle,"Vehicle, Recovered, Truck",Open or Active,Southern,37.77385630397913,-122.40528580238285


In [11]:
# result_renamed.loc[result_renamed['Datetime'] > "2020-02-01" ]

In [12]:
# result_renamed.groupby(['Week', 'Category']).count()

In [13]:
result_sorted = result_renamed.sort_values(by=['Datetime']).reset_index()
result_sorted.head()

,index,Datetime,Time,Year,Week,Category,Description,Resolution,Police_Dist,Lat,Lon
0,22421,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Larceny Theft,"Theft, From Locked Vehicle, >$950",Open or Active,Mission,37.76303086932015,-122.4262284796692
1,15457,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Larceny Theft,"Theft, From Unlocked Vehicle, >$950",Open or Active,Northern,37.7814283707046,-122.43232898312252
2,21319,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Larceny Theft,"Theft, Other Property, >$950",Open or Active,Northern,37.78713043316287,-122.4250469538071
3,12403,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Larceny Theft,"Theft, Other Property, >$950",Open or Active,Southern,37.78179876714438,-122.39834110837147
4,21636,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Lost Property,Lost Property,Open or Active,Mission,37.76505133632968,-122.41966897380142


In [14]:
result_sorted.drop(['index'],axis=1)

,Datetime,Time,Year,Week,Category,Description,Resolution,Police_Dist,Lat,Lon
0,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Larceny Theft,"Theft, From Locked Vehicle, >$950",Open or Active,Mission,37.76303086932015,-122.4262284796692
1,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Larceny Theft,"Theft, From Unlocked Vehicle, >$950",Open or Active,Northern,37.7814283707046,-122.43232898312252
2,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Larceny Theft,"Theft, Other Property, >$950",Open or Active,Northern,37.78713043316287,-122.4250469538071
3,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Larceny Theft,"Theft, Other Property, >$950",Open or Active,Southern,37.78179876714438,-122.39834110837147
4,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Lost Property,Lost Property,Open or Active,Mission,37.76505133632968,-122.41966897380142
...,...,...,...,...,...,...,...,...,...,...
21810,2020-03-03T20:00:00.000,20:00,2020,Tuesday,Robbery,"Robbery, W/ Force",Open or Active,Northern,37.78392473449428,-122.42777887211062
21811,2020-03-03T20:30:00.000,20:30,2020,Tuesday,Traffic Collision,"Traffic Collision, Hit & Run, Property Damage",Cite or Arrest Adult,Southern,37.786851714583754,-122.40126150328634
21812,2020-03-03T20:30:00.000,20:30,2020,Tuesday,Assault,"Assault, Aggravated, W/ Other Weapon",Cite or Arrest Adult,Southern,37.786851714583754,-122.40126150328634
21813,2020-03-03T20:30:00.000,20:30,2020,Tuesday,Traffic Violation Arrest,Traffic Violation Arrest,Cite or Arrest Adult,Southern,37.786851714583754,-122.40126150328634


In [15]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
database = client['Police_reports']
collection = database['Incidents']

# collection.insert_many(result_sorted.to_dict('records'))

In [16]:
# data = pd.DataFrame(list(collection.find({"Category": "Larceny Theft"})))
# data = pd.DataFrame(list(collection.find()))
# data = loads(dumps(collection.find()))
data = collection.find_one()
data

{'_id': ObjectId('5e41c0fa5a30f27f733bdb42'),
 'index': 8693,
 'Datetime': '2020-01-01T00:00:00.000',
 'Time': '00:00',
 'Year': '2020',
 'Week': 'Wednesday',
 'Category': 'Larceny Theft',
 'Description': 'Theft, From Locked Vehicle, >$950',
 'Resolution': 'Open or Active',
 'Police_Dist': 'Northern',
 'Lat': '37.774598784465354',
 'Lon': '-122.42588849947774'}